In [11]:
from collections import defaultdict

import pandas as pd
import torch

import expt

In [16]:
runs_dir = "/dfs/scratch1/ranjanr/runs/cs336/2024-04-28"
df = expt.scan(runs_dir)
df

  0%|          | 0/19 [00:00<?, ?it/s]

,runs_dir,dev,lm_size,vocab_size,context_length,d_model,d_ff,num_layers,num_heads,attn_pdrop,...,batch_size,num_train_steps,backend,device,ddp,rank,local_rank,world_size,done,store_net
1714360289810038297,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,small,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,0,0,2,True,NaN
1714360289859295002,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,small,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,1,1,2,True,NaN
1714360460658588621,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,small,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,0,0,0,1,True,NaN
1714363430967395833,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,small,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,0,0,2,True,0
1714363430978314083,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,small,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,1,1,2,True,0
1714363441182727006,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,medium,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,1,1,2,True,0
1714363441213616747,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,medium,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,0,0,2,True,0
1714363456968391269,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,large,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,0,0,2,True,0
1714363457018651369,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,large,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,1,1,2,True,0
1714363472889436693,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,xl,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,0,0,2,True,0


In [6]:
lm_size = "large"
df.query('lm_size == @lm_size')

,runs_dir,dev,lm_size,vocab_size,context_length,d_model,d_ff,num_layers,num_heads,attn_pdrop,...,batch_size,num_train_steps,backend,device,ddp,rank,local_rank,world_size,done,store_net
1714363456968391269,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,large,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,0,0,2,True,0
1714363457018651369,/dfs/scratch1/ranjanr/runs/cs336/2024-04-28,0,large,10000,128,None,None,None,None,0.0,...,16,10,nccl,cuda,1,1,1,2,True,0


In [15]:
raw = defaultdict(dict)
for lm_size in [
    "small",
    "medium",
    "large",
    "xl",
]:
    step_times = []
    comm_times = []
    for rank in range(2):
        ts = df.query("lm_size == @lm_size and rank == @rank").index[-1]
        store = expt.PathDict(f"{runs_dir}/{ts}")
        step_times += store["step_times"][-5:]
        comm_times += store["comm_times"][-5:]
    step_time = sum(step_times) / len(step_times)
    comm_time = sum(comm_times) / len(comm_times)
    raw[lm_size] = {
        "step time (s)": step_time,
        "comm time (s)": comm_time,
        "comm/step (%)": comm_time/step_time * 100,
    }
pd.DataFrame(raw).style.background_gradient(axis=1).format("{:.3f}")

,small,medium,large,xl
step time (s),0.149,0.479,1.054,2.140
comm time (s),0.024,0.078,0.172,0.357
comm/step (%),16.472,16.366,16.296,16.686


## below is for multi-node (latest runs); should've logged local_world_size

In [17]:
raw = defaultdict(dict)
for lm_size in [
    "small",
    "medium",
    "large",
    "xl",
]:
    step_times = []
    comm_times = []
    for rank in range(2):
        ts = df.query("lm_size == @lm_size and rank == @rank").index[-1]
        store = expt.PathDict(f"{runs_dir}/{ts}")
        step_times += store["step_times"][-5:]
        comm_times += store["comm_times"][-5:]
    step_time = sum(step_times) / len(step_times)
    comm_time = sum(comm_times) / len(comm_times)
    raw[lm_size] = {
        "step time (s)": step_time,
        "comm time (s)": comm_time,
        "comm/step (%)": comm_time/step_time * 100,
    }
pd.DataFrame(raw).style.background_gradient(axis=1).format("{:.3f}")

,small,medium,large,xl
step time (s),0.676,2.255,5.569,10.347
comm time (s),0.568,1.918,4.834,8.892
comm/step (%),84.040,85.048,86.789,85.938
